In [2]:
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Hannanum
import MeCab

#윈도우 mecab 불러오기

from konlpy.tag import Okt
import re

#형태소 분석을 위한 객체 생성
kkma=Kkma()
komoran=Komoran()
hannanum=Hannanum()
#윈도우에서 Mecab생성
mecab=MeCab.Tagger()
okt=Okt()

In [3]:
#여가 딕셔너리 검색어 생성(검색/출력용)
leisure_dict_search=[
    #문화예술관람활동
    ['미술_전시회_관람','사진_전시회_관람','건축_전시회_관람','디자인_전시회_관람',
    '박물관_관람',
    '클래식_음악_연주회_관람','오페라_음악_연주회_관람',
    '국악_전통공연_관람','민속놀이_공연_관람',
    '연극공연_관람','뮤지컬_관람',
    '무용_공연_관람',
    '영화_관람',
    '쇼_공연_관람','콘서트_공연_관람','마술_쇼_공연_관람'],
    #문화예술참여활동
    ['문학_행사_참여',
    '글짓기_참여' ,'독서_토론_참여',
    '그림_활동_참여','서예_활동_참여','조각_활동_참여','디자인_활동_참여','도예_활동_참여','만화_활동_참여',
    '악기_연주_참여','노래교실 참여',
    '춤_참여','무용_참여','발레_참여','한국무용_참여','현대무용_참여','방송댄스_참여','스트릿댄스_참여','비보잉_참여',
    '사진_촬영',
    '사물놀이_참여','줄타기_참여'],
    #스포츠관람활동
    ['축구_경기장_방문_관람' ,'야구_경기장_방문_관람','농구_경기장_방문_관람','배구_경기장_방문_관람',
    '축구_경기_TV_관람','야구_경기_TV_관람','농구_경기_TV_관람','배구_경기_TV_관람',
    '태권도_경기_관람','유도_경기_관람','합기도_경기_관람','검도_경기_관람','권투_경기_관람',
    'e-스포츠_경기_관람'],
    #스포츠참여활동
    ['농구_참여','배구_참여','야구_참여','축구_참여','족구_참여',
    '테니스_참여','스쿼시_참여',
    '당구_참여','포켓볼_참여',
    '볼링_참여','탁구_참여',
    '골프_참여',
    '수영_참여' ,
    '윈드서핑_참여','수상스키_참여','스킨스쿠버다이빙_참여','래프팅_참여','요트_참여',
    '스노보드_참여','스키_참여',
    '아이스스케이트_참여','아이스하키_참여',
    '헬스_참여','보디빌딩_참여','에어로빅_참여',
    '요가_참여','필라테스_참여','태보_참여',
    '배드민턴_참여','줄넘기_참여','맨손_체조_참여','스트레칭_참여','훌라후프_참여',
    '육상_참여','조깅_참여','빠르게_걷기_참여',
    '태권도_참여','유도_참여','합기도_참여','검도_참여','권투_참여',
    '탱고_참여','왈츠_참여','자이브_참여','맘보_참여','폴카_참여','차차차_참여',
    '사이클링_참여','자전거_참여',
    '인라인_스케이트_참여',
    '승마_참여','암벽등반_참여','철인삼종경기_참여','서바이벌_참여'],
    #관광활동
    ['문화유적_방문',
    '자연명승_방문','풍경_관람',
    '삼림욕',
    '국내캠핑',
    '해외여행',
    '소풍','야유회',
    '온천','해수욕',
    '유람선_타기',
    '테마파크_가기','놀이공원_가기','동물원_가기','식물원_가기',
    '지역축제_참가',
    '자동차_드라이브'],
    #취미오락활동
    ['콜렉션','스크랩북',
    '십자수','비즈공예', 'DIY','꽃꽂이',
    '요리하기','다도',
    '반려동물_키우기',
    '노래방_가기',
    '집_인테리어','자동차_인테리어',
    '등산',
    '낚시',
    '홈페이지_관리','블로그_관리',
    '인터넷_검색','1인_미디어_제작','SNS',
    '온라인_게임','모바일_게임','콘솔_게임',
    '보드게임','퍼즐','큐브_맞추기',
    '바둑','장기','체스',
    '경마','경륜','카드놀이','고스톱','마작','복권구입',
    '쇼핑','외식',
    '음주',
    '독서','웹소설',
    '만화보기','웹툰보기',
    '피부관리','헤어관리','네일아트','마사지','성형',
    '어학_공부','기술_공부','자격증_공부',
    '방탈출_카페_체험','VR_카페_체험','낚시카페_체험',
    '원예','화단_가꾸기'],
    #휴식활동
    ['산책','걷기',
    '목욕','사우나','찜질방',
    '낮잠',
    'TV_시청',
    '모바일_컨텐츠','동영상_시청',
    '라디오_청취','팟캐스트_청취',
    '음악_감상',
    '신문_보기','잡지_보기',
    '아무것도_안_하기'],
    #사회 및 기타활동
    ['사회봉사',
    '종교활동',
    '클럽_가기','나이트_가기','디스코_가기','캬바레_가기',
    '가족_방문','친지_방문',
    '잡담','통화하기','문자보내기',
    '계모임','동창회','사교파티',
    '친구_만남','이성교제','미팅','소개팅',
    '동호회_모임']
]

In [4]:
fread=open("./Leisures/Text/미술_전시회_관람.txt", 'r',encoding='UTF8')
templist=fread.read()

In [4]:
templist

'지난주 평일 오후 방배동 근처에 미팅이 두군데나 잡혔어서 점심 먹고 서둘러움직였었는데요. 이거 웬걸, 첫번째 일정이 거래처쪽의 사정으로급하게 취소되면서, 두번째 일정까지시간이 2시간 가량 붕~ 떠 버렸어요. 회사로 돌아갈수도 없고, 카페를 갈까PC방을 갈까.. 혼자 어딜 가지 10여분정도 방황하다가마침 근처에 예술의전당이 있어서평소 잘 하지도 않던 전시회 관람을 해보기로 했어요\u200b\u200b 예술의전당은 학창시절에 단체로 와보고 첨인거 같은데, 평일 오후인데도 의외로관람객들이 많은 편이어서 놀랐어요다들 문화생활 하고 사는구나...ㅎㅎ\u200b원래는 로멘틱코메디 오페라를 보려다가 시간이안 맞아서 미술전시회를 선택했는데, 그렇게 어려운 주제는 아니고 목판화 전시회길래그냥 힐링하는 마음으로 가볍게 볼 수 있겠다싶었죠\u200b\u200b \u200b아무래도 많은 사람들이 모일 수 있는 장소이니만큼발열체크, QR체크, 방명록 작성까지 입장하려면 방역수칙을 따라야 합니다 \u200b 이 목판화 전시회 이름은신비로운 블록버스터 판화의 세계나무, 그림이 되다\u200b블록버스터는 대형 판화 작품이 많아서이름 붙여 진 것이고나무 그림이 되다, 말 그대로나무(목)가 회화가 되다..뭐 이런 뜻\u200b관람료는 만원이었는데,만18세이하까지는 무료,만 65세이상도 무료청소년과 노인분들은 무료로 관람할 수 있는 전시회에요\u200b \u200b입구쪽에 포토존이 있었는데누가 찍어주는 것도 아니고 셀카를 찍기도 챙피해서 그냥 구경만 해봤는데요 2018년 남북정상회담 장면을 재현한거라 합니다\u200b\u200b \u200b입장료 안내를 보면청소년, 노인분들 외에도대학생할인이나 여러 할인혜택들이 있네요예술의전당 전시회 홈페이지에도자세한 내용이 나와 있어요\u200b\u200b \u200b전시회장이 2층부터 3층까지 굉장히 넓었는데대략 700점 정도 작품이 전시되어 있습니다. \u200b사실 미술쪽은 영 문외한이라 그림 볼 줄 도 모르고즐길줄도 모르고 살았는데다행히 어려운 주제

In [8]:
def mecab_nouns(text,obj):
    nouns = []
    obj_list=obj.split("_")
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 일반명사, 고유명사, 동사, 형용사, 어근 추출
    for token in temp:
        if token[0] in obj_list: #자기 자신을 검색어로 가졌기 때문에 제외시킴
            continue
        if token[1] == "NNG" or token[1] == "NNP" or token[1] == "VV" or token[1] == "VA" or token[1] == "XR":
            nouns.append(token[0])
        
    return nouns

def mecab_morphs(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs.append(token[0])
    
    return morphs

def mecab_pos(text):
    pos = []
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    pos = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    return pos

In [9]:
tempresult=mecab_nouns(templist,"미술_전시회_관람")

In [11]:
f = open("./test.txt", 'w',encoding='UTF8')
f.write(str(tempresult))
f.close()

In [10]:
import nltk

In [13]:
type(tempresult)

list

In [11]:
text=nltk.Text(tempresult,name='NMSC')
print(len(set(text.tokens)))#중복을 제외한 개수

21514


In [12]:
print(text.vocab().most_common(200))

[('작품', 6803), ('있', 6384), ('전시', 3660), ('하', 3329), ('보', 2429), ('그림', 2342), ('작가', 2261), ('좋', 2148), ('같', 2010), ('미술관', 1914), ('되', 1716), ('사진', 1437), ('많', 1350), ('전', 1349), ('시간', 1340), ('없', 1335), ('생각', 1251), ('아트', 1224), ('예술', 1143), ('가', 1088), ('아이', 1074), ('사람', 1045), ('현대', 913), ('공간', 891), ('때', 880), ('이번', 872), ('서울', 869), ('층', 842), ('찍', 786), ('들', 734), ('다양', 729), ('갤러리', 729), ('받', 720), ('표현', 707), ('알', 706), ('느낌', 649), ('감상', 642), ('했', 636), ('학원', 634), ('볼', 631), ('곳', 606), ('해', 605), ('속', 600), ('가능', 600), ('모습', 590), ('만들', 585), ('문화', 580), ('말', 554), ('한국', 552), ('대한', 524), ('마음', 515), ('진행', 512), ('통해', 505), ('설명', 502), ('시작', 494), ('예약', 471), ('길', 461), ('보이', 454), ('무료', 450), ('세계', 448), ('일', 448), ('친구', 442), ('화가', 425), ('입장', 422), ('눈', 421), ('소개', 416), ('작업', 416), ('그리', 407), ('영상', 400), ('만나', 391), ('앞', 388), ('주', 385), ('자신', 385), ('이야기', 382), ('맞', 375), ('큰', 371), ('장소', 364), ('